In [1]:
import pandas as pd
df = pd.read_csv('./data/TRBUSDT_5m.csv')
df.head()

,time,open,high,low,close,volume
0,2023-09-01 00:00:00,14.786,14.881,14.785,14.857,38674.7
1,2023-09-01 00:05:00,14.857,14.884,14.820,14.846,13709.4
2,2023-09-01 00:10:00,14.843,14.864,14.810,14.860,12421.1
3,2023-09-01 00:15:00,14.861,14.993,14.860,14.987,41842.2
4,2023-09-01 00:20:00,14.989,15.060,14.980,14.980,33451.1


In [2]:
import pandas_ta as ta
df['EMA50'] = ta.ema(df['close'], length=50)
df['EMA100'] = ta.ema(df['close'], length=100)
df['EMA150'] = ta.ema(df['close'], length=150)

backrollingN = 10
df['slopeEMA50'] = df['EMA50'].diff(periods=1)
df['slopeEMA100'] = df['EMA100'].diff(periods=1)
df['slopeEMA150'] = df['EMA150'].diff(periods=1)

df['slopeEMA50'] = df['slopeEMA50'].rolling(backrollingN).mean()
df['slopeEMA100'] = df['slopeEMA100'].rolling(backrollingN).mean()
df['slopeEMA150'] = df['slopeEMA150'].rolling(backrollingN).mean()

df['ATR']= df.ta.atr()
#help(ta.adx)

In [3]:
#dropna
df = df.dropna()
df.head()

,time,open,high,low,close,volume,EMA50,EMA100,EMA150,slopeEMA50,slopeEMA100,slopeEMA150,ATR
159,2023-09-01 13:15:00,14.754,14.803,14.725,14.799,20306.7,14.855889,14.917422,14.977647,-0.002301,-0.002394,-0.002401,0.066515
160,2023-09-01 13:20:00,14.798,14.966,14.780,14.920,66932.2,14.858403,14.917473,14.976883,-0.001537,-0.002006,-0.002141,0.075050
161,2023-09-01 13:25:00,14.920,14.924,14.832,14.908,41028.1,14.860348,14.917285,14.975971,-0.001112,-0.001782,-0.001990,0.076261
162,2023-09-01 13:30:00,14.907,14.907,14.835,14.874,16193.0,14.860884,14.916428,14.974620,-0.000801,-0.001612,-0.001873,0.076028
163,2023-09-01 13:35:00,14.872,14.880,14.832,14.863,12712.9,14.860967,14.915370,14.973142,-0.000680,-0.001535,-0.001818,0.074026


In [4]:
#set time to index

df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df.head()

C:\Users\hiban\AppData\Local\Temp\ipykernel_5916\1852176384.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['time'])


,open,high,low,close,volume,EMA50,EMA100,EMA150,slopeEMA50,slopeEMA100,slopeEMA150,ATR
time,,,,,,,,,,,,
2023-09-01 13:15:00,14.754,14.803,14.725,14.799,20306.7,14.855889,14.917422,14.977647,-0.002301,-0.002394,-0.002401,0.066515
2023-09-01 13:20:00,14.798,14.966,14.780,14.920,66932.2,14.858403,14.917473,14.976883,-0.001537,-0.002006,-0.002141,0.075050
2023-09-01 13:25:00,14.920,14.924,14.832,14.908,41028.1,14.860348,14.917285,14.975971,-0.001112,-0.001782,-0.001990,0.076261
2023-09-01 13:30:00,14.907,14.907,14.835,14.874,16193.0,14.860884,14.916428,14.974620,-0.000801,-0.001612,-0.001873,0.076028
2023-09-01 13:35:00,14.872,14.880,14.832,14.863,12712.9,14.860967,14.915370,14.973142,-0.000680,-0.001535,-0.001818,0.074026


In [5]:
#rename columns
df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

In [6]:
import numpy as np
conditions = [
    ( (df['EMA50']<df['EMA100']) & (df['EMA100']<df['EMA150']) & (df['slopeEMA50']<0) & (df['slopeEMA100']<0) & (df['slopeEMA150']<0) ),
    ( (df['EMA50']>df['EMA100']) & (df['EMA100']>df['EMA150']) & (df['slopeEMA50']>0) & (df['slopeEMA100']>0) & (df['slopeEMA150']>0) )
           ]
choices = [1, 2]#1: buy, 2: sell
df['EMAsignal'] = np.select(conditions, choices, default=0)

In [7]:
TotSignal = [0] * len(df)
for row in range(0, len(df)):
    TotSignal[row] = 0
    if df.EMAsignal[row]==1 and df.Open[row]>df.EMA50[row] and df.Close[row]<df.EMA50[row]:
        TotSignal[row]=1
    if df.EMAsignal[row]==2 and df.Open[row]<df.EMA50[row] and df.Close[row]>df.EMA50[row]:
        TotSignal[row]=2

df['TotSignal']=TotSignal

In [8]:
df['TotSignal'].value_counts()

0    13550
2      154
1      137
Name: TotSignal, dtype: int64

In [9]:
from backtesting import Strategy
from backtesting import Backtest

def SIGNAL(df,col_name):
    return df[col_name]

class Scalping_Strategy(Strategy):
    atr_sl_rate = 1.3
    TPSL_rate = 1.5
    profit_protect_rate = 0.5
    profit_protect_threshold = 0.5
    init_cash = 100

    def init(self):
        super().init()
        self.signal = self.I(SIGNAL, self.data, 'TotSignal')

    def next(self):
        super().next()

        slatr = self.atr_sl_rate * self.data.ATR[-1]
        #if strategy.profit_ > initial_balance * profit_protect_threshold
        protect_enable = self.equity >= self.init_cash * self.profit_protect_threshold

        if self.signal == 2 :
            sl_price = self.data.Close[-1] - slatr
            tp_price = self.data.Close[-1] + self.TPSL_rate * slatr
            if protect_enable:
                save_size = self.equity * self.profit_protect_rate
                self.buy(sl=sl_price, tp=tp_price, size=int(save_size))
            else:
                self.buy(sl=sl_price, tp=tp_price)

        if self.signal == 1 :
            sl_price = self.data.Close[-1] + slatr
            tp_price = self.data.Close[-1] - self.TPSL_rate * slatr
            if protect_enable:
                save_size = self.equity * self.profit_protect_rate
                self.sell(sl=sl_price, tp=tp_price, size=int(save_size))
            else:
                self.sell(sl=sl_price, tp=tp_price)

bt = Backtest(df, Scalping_Strategy, cash=100, commission=.00002, margin=.05)
stat = bt.run()
stat

Start                     2023-09-01 13:15:00
End                       2023-10-19 14:35:00
Duration                     48 days 01:20:00
Exposure Time [%]                    5.144137
Equity Final [$]                    98.538451
Equity Peak [$]                    347.175767
Return [%]                          -1.461549
Buy & Hold Return [%]              318.859382
Return (Ann.) [%]                    65.71205
Volatility (Ann.) [%]       9292535332.322052
Sharpe Ratio                              0.0
Sortino Ratio                        0.248261
Calmar Ratio                         0.769544
Max. Drawdown [%]                  -85.390902
Avg. Drawdown [%]                  -15.201626
Max. Drawdown Duration       30 days 14:55:00
Avg. Drawdown Duration        1 days 18:39:00
# Trades                                  112
Win Rate [%]                            43.75
Best Trade [%]                       4.322587
Worst Trade [%]                     -2.207084
Avg. Trade [%]                    

In [10]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stat['_equity_curve'].index, y=stat['_equity_curve']['Equity'])])
fig.show()

In [11]:
%%time
stats = bt.optimize(atr_sl_rate = np.arange(0.5, 2.6, 0.2).tolist(),
                    TPSL_rate = np.arange(0.5, 2.6, 0.2).tolist(),
                    profit_protect_rate = np.arange(0.1, 1.0, 0.1).tolist(),
                    profit_protect_threshold = np.arange(0.1, 1.0, 0.1).tolist(),
                    maximize='Equity Final [$]')
stats

C:\Users\hiban\AppData\Roaming\Python\Python310\site-packages\backtesting\backtesting.py:1488: UserWarning:

Searching for best of 9801 configurations.



  0%|          | 0/33 [00:00<?, ?it/s]

CPU times: total: 29min 58s
Wall time: 50min 11s


Start                     2023-09-01 13:15:00
End                       2023-10-19 14:35:00
Duration                     48 days 01:20:00
Exposure Time [%]                    3.525757
Equity Final [$]                  1622.261292
Equity Peak [$]                   1622.261292
Return [%]                        1522.261292
Buy & Hold Return [%]              318.859382
Return (Ann.) [%]          21075864370.687023
Volatility (Ann.) [%]     1039513656110269...
Sharpe Ratio                              0.0
Sortino Ratio                121661809.333677
Calmar Ratio                 344498460.193181
Max. Drawdown [%]                  -61.178399
Avg. Drawdown [%]                   -15.41772
Max. Drawdown Duration        6 days 16:05:00
Avg. Drawdown Duration        0 days 12:20:00
# Trades                                   50
Win Rate [%]                             72.0
Best Trade [%]                       3.568557
Worst Trade [%]                     -2.640395
Avg. Trade [%]                    

In [12]:
stats.keys()

Index(['Start', 'End', 'Duration', 'Exposure Time [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Return [%]', 'Buy & Hold Return [%]',
       'Return (Ann.) [%]', 'Volatility (Ann.) [%]', 'Sharpe Ratio',
       'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]',
       'Avg. Drawdown [%]', 'Max. Drawdown Duration', 'Avg. Drawdown Duration',
       '# Trades', 'Win Rate [%]', 'Best Trade [%]', 'Worst Trade [%]',
       'Avg. Trade [%]', 'Max. Trade Duration', 'Avg. Trade Duration',
       'Profit Factor', 'Expectancy [%]', 'SQN', '_strategy', '_equity_curve',
       '_trades'],
      dtype='object')

In [13]:
stats._strategy

<Strategy Scalping_Strategy(atr_sl_rate=2.3,TPSL_rate=0.7,profit_protect_rate=0.7000000000000001,profit_protect_threshold=0.1)>

In [14]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stats['_equity_curve'].index, y=stats['_equity_curve']['Equity'])])
fig.show()

In [15]:
#heatmap plotly

import plotly.graph_objects as go

#trades
fig = go.Figure(data=go.Heatmap(
                     z=stats['_trades']['PnL'],
                     x=stats['_trades']['EntryTime'],
                     y=stats['_trades']['EntryPrice'],
                     hoverongaps = False))
fig.show()